## Setup

#### Libraries

In [1]:
import pandas as pd
import os
import mysql.connector
from mysql.connector import Error

#### Environnement

In [2]:
DATABASE_HOST = 'localhost'
DATABASE_NAME = 'inventory'
DATABASE_ADMIN_USERNAME = 'root'
DATABASE_ADMIN_PASSWORD = 'tofer006007'

## Extraction

#### Loading from Excel

In [3]:
# Read all csv files in /data folder
data_frames = []
for file in os.listdir('data'):
    if file.endswith('.csv'):
        df = pd.read_csv(f'data/{file}')
        df['date_stock'] = file.split('_')[1].split('.')[0]
        data_frames.append(df)


# Concatenate all dataframes into a single dataframe
stock_df = pd.concat(data_frames, ignore_index=True)

print(stock_df.head(5).to_string())

   RM# Location    Lot #          Product Name Nature of Product  # of boxes  Packages / Box  Units / Package    Units Total (un)  Label grams Total (g)  Total (KG)  date_stock THC % Réservé Retail | Réservé Wholesale
0  110       A1      426              Medellin           Popcorn         NaN             NaN              NaN  22140.0      22140          1.0   22139.6      22.140  2024-04-30   NaN                                NaN
1  110       A1  246261H  Cherry Noir/Mandarin              Hash         NaN             NaN              NaN   3780.0       3780          1.0    3780.0       3.780  2024-04-30   NaN                                NaN
2  110       A1      426              Medellin             Shake         NaN             NaN              NaN  52099.0      52099          1.0   52098.8      52.099  2024-04-30   NaN                                NaN
3  110       A2    MIXED           Sage N Sour         RND|Shred         NaN             NaN              NaN   1392.0       139

## Transformations

### Initial Manipulation

#### Column Manipulations

##### Columns Names

In [4]:
print(pd.Series(stock_df.columns).to_string())

0                                    RM#
1                               Location
2                                  Lot #
3                           Product Name
4                      Nature of Product
5                             # of boxes
6                         Packages / Box
7                        Units / Package
8                                  Units
9                             Total (un)
10                           Label grams
11                             Total (g)
12                            Total (KG)
13                            date_stock
14                                 THC %
15    Réservé Retail | Réservé Wholesale


In [5]:
# Drop columns
stock_df.drop(columns=['Units'], inplace=True)

# Rename columns
stock_df.rename(columns={'RM#': 'room',
                         'Location': 'location',
                         'Lot #': 'batch_name',
                         'Product Name': 'lineage_name',
                         'Nature of Product': 'category_name_initial',
                         '# of boxes': 'boxes',
                         'Packages / Box': 'packages_per_box',
                         'Units / Package': 'units_per_package',
                         'Total (un)': 'units',
                         'Label grams':'grams_per_unit',
                         'Total (g)':'weight_g',
                         'Total (KG)':'weight_kg',
                         'Réservé Retail | Réservé Wholesale':'is_retail',
                         'THC %':'thc'}, inplace=True)

# Add column is_rnd and is_qa
stock_df['is_rnd'] = False
stock_df['needs_qa_approval'] = False
stock_df['released_by_qa'] = False

# Add column category_name
stock_df['category_name'] = stock_df['category_name_initial']

print(pd.Series(stock_df.columns).to_string())

0                      room
1                  location
2                batch_name
3              lineage_name
4     category_name_initial
5                     boxes
6          packages_per_box
7         units_per_package
8                     units
9            grams_per_unit
10                 weight_g
11                weight_kg
12               date_stock
13                      thc
14                is_retail
15                   is_rnd
16        needs_qa_approval
17           released_by_qa
18            category_name


##### Columns Data Types

In [6]:
print(stock_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1765 entries, 0 to 1764
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   room                   1765 non-null   int64  
 1   location               1698 non-null   object 
 2   batch_name             1765 non-null   object 
 3   lineage_name           1765 non-null   object 
 4   category_name_initial  1765 non-null   object 
 5   boxes                  694 non-null    float64
 6   packages_per_box       694 non-null    float64
 7   units_per_package      693 non-null    float64
 8   units                  1765 non-null   object 
 9   grams_per_unit         1765 non-null   float64
 10  weight_g               1765 non-null   object 
 11  weight_kg              1765 non-null   float64
 12  date_stock             1765 non-null   object 
 13  thc                    258 non-null    object 
 14  is_retail              184 non-null    object 
 15  is_r

In [7]:
# Set weight_g to decimal(10,2)
stock_df['weight_g'] = stock_df['weight_g'].str.replace(',', '').astype('float64')

In [8]:
print(stock_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1765 entries, 0 to 1764
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   room                   1765 non-null   int64  
 1   location               1698 non-null   object 
 2   batch_name             1765 non-null   object 
 3   lineage_name           1765 non-null   object 
 4   category_name_initial  1765 non-null   object 
 5   boxes                  694 non-null    float64
 6   packages_per_box       694 non-null    float64
 7   units_per_package      693 non-null    float64
 8   units                  1765 non-null   object 
 9   grams_per_unit         1765 non-null   float64
 10  weight_g               1062 non-null   float64
 11  weight_kg              1765 non-null   float64
 12  date_stock             1765 non-null   object 
 13  thc                    258 non-null    object 
 14  is_retail              184 non-null    object 
 15  is_r

#### Values Manipulations

##### is_retail

In [9]:
print(stock_df['is_retail'].head(10).to_string())

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN


In [10]:
# for column is_retail, is value is None, remove row
stock_df = stock_df[stock_df['is_retail'].notnull()]

print(stock_df['is_retail'].head(10).to_string())

1218       Retail
1219       Retail
1220    Wholesale
1221    Wholesale
1222       Retail
1223    Wholesale
1224    Wholesale
1225    Wholesale
1226    Wholesale
1227    Wholesale


In [11]:
# if is_retail is 'Retail', set value to True, else False
stock_df.loc[stock_df['is_retail'] == 'Retail', 'is_retail'] = True
stock_df.loc[stock_df['is_retail'] == 'Wholesale', 'is_retail'] = False

print(stock_df['is_retail'].head(10).to_string())

1218     True
1219     True
1220    False
1221    False
1222     True
1223    False
1224    False
1225    False
1226    False
1227    False


##### others

In [12]:
# In category_name, replace 'RND', 'QA', 'Defect' and '|' with ''
stock_df['category_name'] = stock_df['category_name'].str.replace('RND', '')
stock_df['category_name'] = stock_df['category_name'].str.replace('QA', '')
stock_df['category_name'] = stock_df['category_name'].str.replace('Defect', '')
stock_df['category_name'] = stock_df['category_name'].str.replace('|', '')

# Logical loop to determine if the product is rnd
# Look for the word 'RND' or 'QA' in the category_name
stock_df.loc[stock_df['category_name_initial'].str.contains('RND', case=False), 'is_rnd'] = True
stock_df.loc[stock_df['category_name_initial'].str.contains('QA', case=False), 'needs_qa_approval'] = True
stock_df.loc[stock_df['category_name_initial'].str.contains('Defect', case=False), 'is_defect'] = True

# Set released_by_qa based on needs_qa_approval
stock_df.loc[stock_df['needs_qa_approval'], 'released_by_qa'] = False
stock_df.loc[~stock_df['needs_qa_approval'], 'released_by_qa'] = True


# For column is_retail, select only when value is NaN
stock_df.loc[stock_df['is_retail'].isnull(), 'is_retail'] = None
stock_df.loc[stock_df['is_defect'].isnull(), 'is_defect'] = None

stock_df['boxes'] = stock_df['boxes'].astype('Int64')
stock_df['packages_per_box'] = stock_df['packages_per_box'].astype('Int64')
stock_df['units_per_package'] = stock_df['units_per_package'].astype('Int64')

# For every row, if a value is NaN, replace with None
#stock_df = stock_df.where(pd.notnull(stock_df), None)
stock_df = stock_df.replace({pd.NA: None})

In [13]:
# show as table
print(stock_df.head(20).to_string())

      room location batch_name      lineage_name category_name_initial boxes packages_per_box units_per_package   units  grams_per_unit  weight_g  weight_kg  date_stock     thc is_retail  is_rnd  needs_qa_approval  released_by_qa category_name is_defect
1218   110       A1        426          Medellin               Popcorn    21             1000                 1  22,140             1.0   22139.6     22.140  2024-06-30  27.53%      True   False              False            True       Popcorn      None
1219   110       A1        443          Medellin               Popcorn    20             1000                 1  20,883             1.0   20883.3     20.883  2024-06-30  29.20%      True   False              False            True       Popcorn      None
1220   110       A1        318  Super Lemon Haze       QA|Large Flower     6             1000                 1   6,101             1.0    6101.4      6.101  2024-06-30    None     False   False               True           False  Large F

##### Categories changes

In [14]:
# print all distinct values in category_name
print(pd.Series(stock_df['category_name'].unique()).to_string())

0          Popcorn
1     Large Flower
2            Shake
3    Popcorn/Shake
4            Shred


In [15]:
# in column category_name, replace value 'Hash' with 'Haschich'
stock_df['category_name'] = stock_df['category_name'].str.replace('Hash', 'Haschich')
stock_df['category_name'] = stock_df['category_name'].str.replace('Prérouler', 'Préroulés')

# remove ' 3.5'
stock_df['category_name'] = stock_df['category_name'].str.replace(' 3.5', '')
stock_df['category_name'] = stock_df['category_name'].str.replace(' 28.0', '')
stock_df['category_name'] = stock_df['category_name'].str.replace(' 3.0', '')
stock_df['category_name'] = stock_df['category_name'].str.replace(' 0.75', '')
stock_df['category_name'] = stock_df['category_name'].str.replace(' 0.6', '')
stock_df['category_name'] = stock_df['category_name'].str.replace(' 1.0', '')
stock_df['category_name'] = stock_df['category_name'].str.replace(' 2.0', '')
stock_df['category_name'] = stock_df['category_name'].str.replace(' 15.0', '')
stock_df['category_name'] = stock_df['category_name'].str.replace(' 0.35', '')

In [16]:
# print all distinct values in category_name
print(pd.Series(stock_df['category_name'].unique()).to_string())

0          Popcorn
1     Large Flower
2            Shake
3    Popcorn/Shake
4            Shred


In [17]:
# print distribution
print(stock_df['category_name'].value_counts().to_string())

category_name
Popcorn          62
Large Flower     47
Popcorn/Shake    39
Shake            25
Shred            11


### MySQL

## Queries

In [18]:
# Print category_name = '450'
print(stock_df.head(20).to_string())

      room location batch_name      lineage_name category_name_initial boxes packages_per_box units_per_package   units  grams_per_unit  weight_g  weight_kg  date_stock     thc is_retail  is_rnd  needs_qa_approval  released_by_qa category_name is_defect
1218   110       A1        426          Medellin               Popcorn    21             1000                 1  22,140             1.0   22139.6     22.140  2024-06-30  27.53%      True   False              False            True       Popcorn      None
1219   110       A1        443          Medellin               Popcorn    20             1000                 1  20,883             1.0   20883.3     20.883  2024-06-30  29.20%      True   False              False            True       Popcorn      None
1220   110       A1        318  Super Lemon Haze       QA|Large Flower     6             1000                 1   6,101             1.0    6101.4      6.101  2024-06-30    None     False   False               True           False  Large F